In [2]:
# Install necessary libraries
!pip install transformers
!pip install torch
!pip install pandas
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import pandas as pd

# Load your FASTA file
# Change this path to where your FASTA file is stored in Colab (e.g., uploaded file)
fasta_file = '/content/filtered_cleaned_peptides2.fasta'

def load_fasta(fasta_file):
    """Function to load peptide sequences from a FASTA file"""
    with open(fasta_file, 'r') as file:
        lines = file.readlines()

    peptides = []
    for i in range(1, len(lines), 2):  # Skip the headers (lines[0], lines[2], ...)
        sequence = lines[i].strip()  # Remove any newline characters
        peptides.append(sequence)

    return peptides

# Load the peptides
peptides = load_fasta(fasta_file)

# Display first few peptides
print(peptides[:5])  # Display the first 5 peptides


['TPYDINQML', 'VPLRPMTY', 'KPIVQYDNF', 'GSVVIVGRIVLSGKPA', 'GSVVIVGRIVLSGKPA']


In [4]:
from huggingface_hub import login

# Log in to Hugging Face (this will prompt to enter Hugging Face API token)
login()


In [5]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Load the correct model architecture, change this to the correct PepMLM model identifier
model_name = "TianlaiChen/PepMLM-650M"  # Replace with the correct identifier for PepMLM

# Load the model and tokenizer (use MaskedLM architecture for peptide sequences)
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize peptide sequences (assuming peptides is your list of peptide sequences)
tokenized_data = tokenizer(peptides, padding=True, truncation=True, return_tensors="pt")

# Now you can proceed with fine-tuning this model


config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
peptides = ['TPYDINQML', 'VPLRPMTY', 'KPIVQYDNF', 'GSVVIVGRIVLSGKPA', 'GSVVIVGRIVLSGKPA']
# Tokenize the peptide sequences
encoding = tokenizer(peptides, padding=True, truncation=True, return_tensors="pt")

# Display tokenized output (first peptide for example)
print(encoding['input_ids'][0])


tensor([ 0, 11, 14, 19, 13, 12, 17, 16, 20,  4,  2,  1,  1,  1,  1,  1,  1,  1])


In [7]:
from transformers import AutoModelForMaskedLM

# Load the PepMLM model
model = AutoModelForMaskedLM.from_pretrained("TianlaiChen/PepMLM-650M")


In [8]:
!pip install biopython
from Bio import SeqIO

# Load the FASTA file containing your peptide sequences
fasta_file = "/content/filtered_cleaned_peptides2.fasta"  # Provide your actual file path

def load_fasta(fasta_file):
    """Load peptide sequences from a FASTA file"""
    peptides = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        peptides.append(str(record.seq))  # Add each peptide sequence to the list
    return peptides

# Load your peptides
peptides = load_fasta(fasta_file)

# Display the first few peptides to confirm
print(peptides[:5])  # Show the first 5 peptides to check the data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.5 MB/s eta 0:00:00
['TPYDINQML', 'VPLRPMTY', 'KPIVQYDNF', 'GSVVIVGRIVLSGKPA', 'GSVVIVGRIVLSGKPA']


In [9]:
from transformers import AutoTokenizer

# Load the pre-trained PepMLM tokenizer
tokenizer = AutoTokenizer.from_pretrained("TianlaiChen/PepMLM-650M")

# Tokenize the peptide sequences
def tokenize_peptides(peptides, tokenizer):
    """Function to tokenize the peptide sequences"""
    encoding = tokenizer(peptides, padding=True, truncation=True, return_tensors="pt")
    return encoding

# Tokenize the loaded peptides
tokenized_data = tokenize_peptides(peptides, tokenizer)

# Display the tokenized output of the first peptide for verification
print(tokenized_data['input_ids'][0])  # Show the token IDs for the first peptide


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([ 0, 11, 14, 19, 13, 12, 17, 16, 20,  4,  2,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])


In [11]:
!pip install datasets
from datasets import Dataset

# Create a Hugging Face Dataset from the tokenized peptides
train_dataset = Dataset.from_dict({
    "input_ids": tokenized_data['input_ids'],
    "attention_mask": tokenized_data['attention_mask']
})

# Display the dataset
print(train_dataset[0])  # Show the first tokenized sample


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
{'input_ids': [0, 11, 14, 19, 13, 12, 17, 16, 20, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [12]:
!pip install --upgrade transformers


In [13]:
from transformers import TrainingArguments

# Disable W&B logging
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=1,
    warmup_steps=100,
    report_to="none",  # Disable W&B logging
)


In [14]:
from transformers import Trainer, TrainingArguments, AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling
from datasets import Dataset

# Load the model and tokenizer
model_name = "TianlaiChen/PepMLM-650M"  # Replace this with your model name
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare peptide sequences (assuminG already have the peptides)
peptides = ['TPYDINQML', 'VPLRPMTY', 'KPIVQYDNF', 'GSVVIVGRIVLSGKPA', 'GSVVIVGRIVLSGKPA']  # Example peptides, replace with yours

# Tokenize peptide sequences, specify a max_length to avoid truncation warnings
max_length = 512  # THIS can adjust this based on your sequence lengths
tokenized_data = tokenizer(peptides, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Create a Hugging Face Dataset from the tokenized data
train_dataset = Dataset.from_dict({
    "input_ids": tokenized_data['input_ids'],
    "attention_mask": tokenized_data['attention_mask']
})

# Define a data collator for masked language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,  # This enables MLM
    mlm_probability=0.15  # Percentage of tokens to be masked
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",              # Where to save the model
    per_device_train_batch_size=2,       # Batch size (adjust based on your GPU)
    num_train_epochs=3,                  # Number of epochs to train
    logging_dir="./logs",                # Directory to save logs
    logging_steps=10,                    # How often to log
    save_steps=500,                      # Save checkpoint every 500 steps
    save_total_limit=1,                  # Limit the number of saved checkpoints
    warmup_steps=100,                    # Learning rate warmup
    report_to="none",                    # Disable W&B logging
)

# Initialize Trainer with data collator
trainer = Trainer(
    model=model,                         # The model to fine-tune
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # The dataset to train on
    data_collator=data_collator          # Data collator for MLM
)

# Start fine-tuning
trainer.train()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_pepmlm_model")
tokenizer.save_pretrained("fine_tuned_pepmlm_model")


Step,Training Loss


('fine_tuned_pepmlm_model/tokenizer_config.json',
 'fine_tuned_pepmlm_model/special_tokens_map.json',
 'fine_tuned_pepmlm_model/vocab.txt',
 'fine_tuned_pepmlm_model/added_tokens.json')

In [29]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import os

# Use absolute path to avoid issues with relative paths
model_path = "/content/fine_tuned_pepmlm_model"  # Replace with the actual absolute path

# Ensure no hub interaction
model = AutoModelForMaskedLM.from_pretrained(model_path, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [ ]:
import random
import time
import csv
from datetime import timedelta
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM
import Levenshtein

# Load fine-tuned PepMLM model
model_path = "./pepmlm_finetuned"  # Update this path to your model's path
model = AutoModelForMaskedLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create the MaskedLM pipeline
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer, top_k=5)
fill_mask.model.to("cuda:0")  # Use GPU if available

# Relax duplicate checking (Levenshtein distance)
def is_duplicate(new_peptide, seen_peptides, max_dist=3):
    for peptide in seen_peptides:
        if Levenshtein.distance(new_peptide, peptide) <= max_dist:
            return True
    return False

# Function to generate peptide with confidence scores
def generate_peptide(context="GTGSGKS", min_len=12, max_len=16):
    peptide = context
    scores = []
    target_len = random.randint(min_len, max_len)

    while len(peptide) < target_len:
        masked_input = " ".join(list(peptide)) + f" {tokenizer.mask_token}"
        try:
            predictions = fill_mask(masked_input)
        except Exception:
            break

        best = None
        for pred in predictions:
            token = pred["token_str"]
            score = pred["score"]
            if token.isalpha() and token != "X":
                best = (token, score)
                break

        if best is None:
            break

        token, score = best
        peptide += token
        scores.append(score)

    if len(scores) == 0:
        return peptide, 0.0

    avg_score = sum(scores) / len(scores)
    return peptide, avg_score

# Function to generate multiple peptides
def generate_multiple_peptides(n_target=250, min_len=12, max_len=16, score_threshold=0.005, max_attempts=10000):
    seen = set()
    peptides = []
    start_time = time.time()
    attempts = 0

    contexts = [
        "GTGSGKS",      # Canonical KRAS P-loop
        "SGKSALTI",     # P-loop tail
        "KLVVVGAG",     # ATPase/GTPase fold
        "GPPNVG",       # Common KRAS motif start (Gly-rich)
        "IGAQGE",       # Switch I start
    ]

    csv_path = "generated_peptides.csv"
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Peptide", "Score"])

        while len(peptides) < n_target and attempts < max_attempts:
            context = random.choice(contexts)
            pep, score = generate_peptide(context=context, min_len=min_len, max_len=max_len)
            attempts += 1

            if score >= score_threshold and pep not in seen and not is_duplicate(pep, seen):
                seen.add(pep)
                peptides.append((pep, score))
                writer.writerow([pep, score])

                # Log progress
                elapsed = time.time() - start_time
                avg_time = elapsed / len(peptides)
                eta = timedelta(seconds=(avg_time * n_target - elapsed))
                print(f"[{len(peptides)}/{n_target}] {pep} | Score: {score:.5f} | ETA: {eta}")

    if len(peptides) < n_target:
        print(f"Stopped early: {len(peptides)} unique high-score peptides found in {attempts} attempts.")
    else:
        print(f"Successfully generated {len(peptides)} peptides. Saved to {csv_path}")

    return peptides

# Example run for 250 peptides
peptides = generate_multiple_peptides(n_target=250)
